In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/ViT5/examples


In [2]:
import sys
sys.path.append("")

In [3]:
# !wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
# !unzip 'master.zip'

In [4]:
import glob
import pandas as pd
pd.set_option('display.max_columns', None)
import concurrent.futures
from datasets import *
import datasets
from sklearn.model_selection import train_test_split
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments


# from vncorenlp import VnCoreNLP
import torch 
import nltk
from nltk.tokenize import sent_tokenize



/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-11 11:16:40.631890: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-11 11:16:40.674848: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 11:16:41.309300: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find 

In [5]:
OUTPUT_DIR = 'Vietnam_T5_small_200'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [6]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/mrzaizai2k/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Processing data

In [7]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('data/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('data/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('data/vietnews-master/data/test_tokenized/*')

In [8]:
def read_content(pathfile):
    """
    Input: Path of txt file
    Output: A dictionary has keys 'original' and 'summary'
    """
    with open(pathfile) as f:
      rows  = f.readlines()
      original = ' '.join(''.join(rows[4:]).split('\n'))
      summary = ' '.join(rows[2].split('\n'))
            
    return {'file' : pathfile,
              'original': original, 
              'summary': summary}

In [9]:
read_content(train_paths[0])

{'file': 'data/vietnews-master/data/train_tokenized/006157.txt.seg',
 'original': 'Tập 4 Thần_tượng Âm_nhạc nhí - Vietnam_Idol_Kids 2017 lên sóng tối 2/6 với màn tranh tài của top 8 thí_sinh nữ để lựa_chọn ra 5 tấm vé vào tiếp vòng trong . Trong đó , 3 giám_khảo Isaac - Văn_Mai_Hương - Bích_Phương được quyền lựa_chọn 3 thí_sinh . 2 thí_sinh còn lại sẽ do các giám_khảo khách mời quyết_định . Nữ ca_sĩ Văn_Mai_Hương xúc_động : “ Cô có may_mắn năm nay ngồi ở vị_trí ban giám_khảo , may_mắn hơn là được gặp con . Mỗi khi gặp Hiền , cô tự thấy bản_thân cô rất kém , bởi có lúc cô không trân_trọng cũng như không tin vào bản_thân ... Cảm_ơn con , bởi đôi_khi có những cái cô không bằng con được , đó là sự lạc_quan . Và cô tin , còn rất nhiều người phải học đức_tính lạc_quan này của Hiền . Con hát rất là hay ” . Đồng_tình với ý_kiến của đồng_nghiệp , Bích_Phương cũng xúc_động chia_sẻ : “ Giọt nước_mắt dành cho con là sự khâm_phục chứ không phải là thương_cảm . Cô chưa bao_giờ thấy con buồn . Từ lúc

In [10]:
def get_dataframe(pathfiles):
    with concurrent.futures.ProcessPoolExecutor() as executor:
      data = executor.map(read_content, pathfiles)
    
    # Make blank dataframe
    data_df = list()
    for d in data:
      data_df.append(d)
    data_df = pd.DataFrame(data_df)
    data_df.dropna(inplace = True)
    data_df = data_df.sample(frac=1).reset_index(drop=True)

    return data_df

In [11]:
train_df = get_dataframe(train_paths[:100])

In [12]:
val_df = get_dataframe(val_paths[:100])
test_df = get_dataframe(test_paths[:100])

In [13]:
test_df.head()

,file,original,summary
0,data/vietnews-master/data/test_tokenized/00859...,"Cảnh_sát bang Jharkhand , phía đông Ấn_Độ hôm ...",Dhanu_Bhuiyan bị buộc_tội cưỡng_hiếp thiếu_nữ ...
1,data/vietnews-master/data/test_tokenized/00904...,Tấm cửa_sổ bằng nhựa có kích_thước 58 x 47 cm ...,Trực_thăng CH -53 E của thuỷ_quân_lục_chiến Mỹ...
2,data/vietnews-master/data/test_tokenized/01516...,"Ngày 3/12 , ông Phạm_Văn_Ru , Trưởng_ban Tổ_ch...","Do vẫn mong_muốn được công_tác , bà Phan_Thị_M..."
3,data/vietnews-master/data/test_tokenized/00560...,Ngoại_trưởng Zarif khẳng_định Iran không muốn ...,Ngoại_trưởng Iran_Mohammad_Javad_Zarif dịu giọ...
4,data/vietnews-master/data/test_tokenized/01738...,"Trò_chuyện với chúng_tôi , H. gửi tấm hình hai...",“ Có dịp vui anh_em lại tổ_chức “ bay ” thôi ....


In [14]:
# test_df.to_parquet("data/vietnews/test.parquet", index=False)

In [15]:
# train_df = pd.read_parquet("data/vietnews/train.parquet")
# val_df = pd.read_parquet("data/vietnews/val.parquet")
# test_df = pd.read_parquet("data/vietnews/test.parquet")
# test_df.head()

In [16]:
# import matplotlib.pyplot as plt

# train_df['word_count'] = train_df['summary'].apply(lambda x: len(x.split()))

# # Plot the histogram
# plt.figure(figsize=(10, 6))
# plt.hist(train_df['word_count'], bins=100, edgecolor='black')
# plt.title('Histogram of Number of Words in Sentences')
# plt.xlabel('Number of Words')
# plt.ylabel('Frequency')
# plt.show()

## **Warm-starting RoBERTaShared for BBC XSum**

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


### **Data Preprocessing**


In [17]:
pretrained_model = "google-t5/t5-small"
# pretrained_model = "Falconsai/text_summarization"
tokenizer = AutoTokenizer.from_pretrained("Vietnam_T5_small_200")


In [18]:
train_data =  Dataset.from_pandas(train_df)
val_data =  Dataset.from_pandas(val_df)
test_data =  Dataset.from_pandas(test_df)

In [21]:
batch_size=16 # change to 16 for full training
block_size = 128
max_input_length = 512 #1024
max_target_length = 128 #128

def process_data_to_model_inputs(examples):
    return tokenizer([" ".join(x) for x in examples["original"]])

# def process_data_to_model_inputs(batch):
#     model_inputs = tokenizer(
#         batch["original"],
#         max_length=max_input_length,
#         truncation=True,
#     )
#     labels = tokenizer(
#         batch["summary"], max_length=max_target_length, truncation=True
#     )
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
# train_data = train_data.select(range(32))

train_data_batch = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file","original", "summary"],
)
train_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# val_data = val_data.select(range(16))

val_data_batch = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file", "original", "summary"],
)
val_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask"],
)



Map: 100%|██████████| 100/100 [00:00<00:00, 756.94 examples/s]


In [22]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

lm_dataset = train_data_batch.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]


TypeError: can only concatenate list (not "Tensor") to list

In [ ]:
tokenizer.pad_token_id

0

In [ ]:
train_df.iloc[0]

file        data/vietnews-master/data/train_tokenized/0012...
original    Kịch_bản của kẻ_cướp ngân_hàng Liên_quan đến v...
summary     Liên_quan đến vụ cướp ngân_hàng Agribank ở Thá...
Name: 0, dtype: object

In [ ]:
a = tokenizer(str(train_df["summary"].iloc[0]))
print(str(train_df["summary"].iloc[0]))
a

Liên_quan đến vụ cướp ngân_hàng Agribank ở Thái_Bình , lãnh_đạo Công_an tỉnh Thái_Bình cho biết , trước khi gây án , nghi_phạm đã có kế_hoạch chuẩn_bị rất kỹ , dùng các chiêu để đánh lạc hướng mọi người xung_quanh . 


{'input_ids': [712, 2, 12364, 149, 178, 1069, 123, 690, 105, 2, 2054, 3044, 2161, 103, 135, 672, 2, 9145, 103, 104, 409, 113, 2, 148, 536, 121, 248, 2, 117, 105, 260, 672, 2, 9145, 122, 182, 103, 104, 235, 109, 145, 546, 125, 103, 143, 105, 103, 104, 787, 2, 123, 1632, 127, 119, 564, 2, 113, 121, 5114, 581, 556, 2, 337, 1673, 317, 108, 103, 190, 524, 103, 104, 415, 115, 120, 2186, 111, 141, 445, 1126, 103, 113, 513, 709, 130, 862, 115, 2, 12364, 113, 103, 106, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.convert_ids_to_tokens(train_data_batch[0]["labels"])[:10]

['▁Liên', '<unk>', 'quan', '▁đến', '▁vụ', '▁cướ', 'p', '▁ngâ', 'n', '<unk>']

In [ ]:
# train_data_batch[0]

### **Warm-starting the Encoder-Decoder Model**

In [23]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained(pretrained_model).to(device)

my_config = model.generation_config
print("generation_config", model.generation_config)

generation_config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0
}



### **Fine-Tuning Warm-Started Encoder-Decoder Models**

The `Seq2SeqTrainer` that can be found under [examples/seq2seq/seq2seq_trainer.py](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/seq2seq_trainer.py) will be used to fine-tune a warm-started encoder-decoder model.

Let's download the `Seq2SeqTrainer` code and import the module along with `TrainingArguments`.

We need to add some additional parameters to make `TrainingArguments` compatible with the `Seq2SeqTrainer`. Let's just copy the `dataclass` arguments as defined in [this file](https://github.com/patrickvonplaten/transformers/blob/make_seq2seq_trainer_self_contained/examples/seq2seq/finetune_trainer.py).

Also, we need to define a function to correctly compute the ROUGE score during validation. ROUGE is a much better metric to track during training than only language modeling loss.

In [ ]:
import numpy as np
import evaluate

rouge_score = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import DataCollatorForLanguageModeling
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
features = [train_data_batch[i] for i in range(2)]
data_collator(features)

/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


{'input_ids': tensor([[ 489, 1181,    2,  ...,  103,  117,    1],
        [ 103, 1454,  165,  ...,  322,  107,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[  712,     2, 12364,   149,   178,  1069,   123,   690,   105,     2,
          2054,  3044,  2161,   103,   135,   672,     2,  9145,   103,   104,
           409,   113,     2,   148,   536,   121,   248,     2,   117,   105,
           260,   672,     2,  9145,   122,   182,   103,   104,   235,   109,
           145,   546,   125,   103,   143,   105,   103,   104,   787,     2,
           123,  1632,   127,   119,   564,     2,   113,   121,  5114,   581,
           556,     2,   337,  1673,   317,   108,   103,   190,   524,   103,
           104,   415,   115,   120,  2186,   111,   141,   445,  1126,   103,
           113,   513,   709,   130,   862,   115,     2, 12364,   113,   103,
           106,     1],
        [ 2263,   967,   204,   157,   553,   1

Cool! Finally, we start training.

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
logging_steps = min(2000,len(train_data_batch) // batch_size)
print("logging_steps", logging_steps)

training_args = Seq2SeqTrainingArguments(
    output_dir= OUTPUT_DIR,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    eval_strategy="epoch",
    learning_rate=2e-5,
    # do_train=True,
    # do_eval=True,
    logging_steps=logging_steps,  # set to 2000 for full training
    save_steps=500,  # set to 500 for full training
    eval_steps=750,  # set to 7500 for full training
    warmup_steps=3000,  # set to 3000 for full training
    num_train_epochs=10, #uncomment for full training
    overwrite_output_dir=True,
    optim='adafactor',
    save_total_limit=3,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model,
    args=training_args,
    train_dataset=train_data_batch,
    eval_dataset=val_data_batch,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

logging_steps 12


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,5.525600,5.013842,5.130100,1.110400,4.345400,4.461800
2,5.386600,5.012387,5.280100,1.159200,4.485600,4.609600
3,5.533100,5.010102,5.280100,1.159200,4.485600,4.609600
4,5.374500,5.006131,5.223300,1.159200,4.430900,4.548300
5,5.546100,5.002307,4.946200,1.005100,4.177800,4.312700
6,5.326000,4.996574,4.991400,1.007500,4.173100,4.309000
7,5.543200,4.990853,5.152900,1.161400,4.286900,4.397700
8,5.448200,4.982821,5.022300,0.970900,4.151300,4.234600
9,5.400600,4.973183,5.219900,1.014600,4.291000,4.382300
10,5.455800,4.963674,5.325900,1.043200,4.358900,4.431200


/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=130, training_loss=5.448418279794546, metrics={'train_runtime': 72.9699, 'train_samples_per_second': 13.704, 'train_steps_per_second': 1.782, 'total_flos': 135341801472000.0, 'train_loss': 5.448418279794546, 'epoch': 10.0})

In [ ]:
trainer.save_model(OUTPUT_DIR)

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]


training_args.bin: 100%|██████████| 5.30k/5.30k [00:00<00:00, 14.2kB/s]
events.out.tfevents.1718077729.DESKTOP-H2CRQMR.405568.0: 100%|██████████| 13.0k/13.0k [00:00<00:00, 33.5kB/s]
model.safetensors: 100%|██████████| 242M/242M [00:23<00:00, 10.2MB/s] 
Upload 3 LFS files: 100%|██████████| 3/3 [00:24<00:00,  8.05s/it]


In [ ]:
# !gsutil -m cp -r '/content/training/*' 'gs://kaggle-vbdi-test/training_Data'

### **Evaluation**

Awesome, we finished training our dummy model. Let's now evaluated the model on the test data. We make use of the dataset's handy `.map()` function to generate a summary of each sample of the test data.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, use_fast=False)

# model = EncoderDecoderModel.from_pretrained(OUTPUT_DIR + "/checkpoint-4000")
model = AutoModelForSeq2SeqLM.from_pretrained(OUTPUT_DIR, torch_dtype=torch.bfloat16,).to(device)
print("generation_config", model.generation_config)


# test_data = datasets.load_dataset("xsum", split="test")

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    outputs = model.generate(input_ids, attention_mask=attention_mask,
                             max_length=128,
                        # num_beams=4,
                        # no_repeat_ngram_size=3,
                        # early_stopping=True,
                        )

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["original"])

pred_str = results["pred"]
label_str = results["summary"]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Parameter 'function'=<function generate_summary at 0x7f423895ff40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous 

generation_config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0
}



Map: 100%|██████████| 100/100 [00:08<00:00, 11.36 examples/s]


In [ ]:
rouge_output = rouge_score.compute(predictions=pred_str, references=label_str, use_stemmer=True)
rouge_output = {key: value * 100 for key, value in rouge_output.items()}
rouge_output

{'rouge1': 34.278702846384604,
 'rouge2': 8.099618792816216,
 'rougeL': 22.1942267098939,
 'rougeLsum': 22.140831895853147}

In [ ]:
for i in range(10):
    print('\nPrediction: ',pred_str[i])
    print('Truth: ',label_str[i])
    print('Content: ',test_data[i]['original'])
    


Prediction:  trong bà Mnh vit trong bà Mnh vit trong bà Mnh vit trong bà Mnh vit trong bà Mnh vit trong bà Mnh vit trong bà trong bà th công_b trên mt din
Truth:  Lần đầu_tiên kể từ khi bị bắt tại Canada tháng 12-2018 , giám_đốc tài_chính toàn_cầu công_ty Huawei , bà Mạnh_Vãn_Chu , gửi thư tới 188.000 nhân_viên công_ty này , cảm_ơn sự động_viên của họ . 
Content:  Theo đài CNN , trong bản_sao bức thư họ nhận được , bà Mạnh cảm_ơn gần 200.000 nhân_viên Huawei toàn_cầu vì đã hết_lòng ủng_hộ , động_viên bà , trao cho bà " sức_mạnh " trong thời_gian qua . Bà Mạnh_Vãn_Chu vẫn đang chờ_đợi phiên xử tại toà Canada để quyết_định việc có bị dẫn_độ qua Mỹ xét_xử không . Chính_quyền Mỹ cáo_buộc bà Mạnh vi_phạm các lệnh trừng_phạt của Mỹ với Iran . " Cứ mỗi khi phiên xét_xử kết_thúc , tôi lại thấy các nhân_viên của Huawei đứng chờ trắng đêm chỉ để theo_dõi vụ_việc của tôi theo các múi thời_gian cách xa nhau " , bà Mạnh viết trong bức thư được công_bố trên một diễn_đàn mạng nội_bộ của công_ty Huaw

In [ ]:
test_data[0]['file']

'data/vietnews-master/data/test_tokenized/004984.txt.seg'